In [1]:
#importing the dataset
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
#doing train and test split
size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [3]:
#reviews are stored as integers
print('reviews - ')
print(X_train[10])

#labels are stored as integers, 0 for negative, 1 for positive
print('label - ')
print(y_train[10])

reviews - 
[1, 785, 189, 438, 47, 110, 142, 7, 6, 2, 120, 4, 236, 378, 7, 153, 19, 87, 108, 141, 17, 1004, 5, 2, 883, 2, 23, 8, 4, 136, 2, 2, 4, 2, 43, 1076, 21, 1407, 419, 5, 2, 120, 91, 682, 189, 2818, 5, 9, 1348, 31, 7, 4, 118, 785, 189, 108, 126, 93, 2, 16, 540, 324, 23, 6, 364, 352, 21, 14, 9, 93, 56, 18, 11, 230, 53, 771, 74, 31, 34, 4, 2834, 7, 4, 22, 5, 14, 11, 471, 9, 2, 34, 4, 321, 487, 5, 116, 15, 2, 4, 22, 9, 6, 2286, 4, 114, 2679, 23, 107, 293, 1008, 1172, 5, 328, 1236, 4, 1375, 109, 9, 6, 132, 773, 2, 1412, 8, 1172, 18, 2, 29, 9, 276, 11, 6, 2768, 19, 289, 409, 4, 2, 2140, 2, 648, 1430, 2, 2, 5, 27, 3000, 1432, 2, 103, 6, 346, 137, 11, 4, 2768, 295, 36, 2, 725, 6, 3208, 273, 11, 4, 1513, 15, 1367, 35, 154, 2, 103, 2, 173, 7, 12, 36, 515, 3547, 94, 2547, 1722, 5, 3547, 36, 203, 30, 502, 8, 361, 12, 8, 989, 143, 4, 1172, 3404, 10, 10, 328, 1236, 9, 6, 55, 221, 2989, 5, 146, 165, 179, 770, 15, 50, 713, 53, 108, 448, 23, 12, 17, 225, 38, 76, 4397, 18, 183, 8, 81, 19, 12, 45, 

In [4]:
#to get the reviews into words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('reviews with words - ')
print([id2word.get(i, ' ') for i in X_train[10]])
print('label - ')
print(y_train[10])

reviews with words - 
['the', 'clear', 'fact', 'entertaining', 'there', 'life', 'back', 'br', 'is', 'and', 'show', 'of', 'performance', 'stars', 'br', 'actors', 'film', 'him', 'many', 'should', 'movie', 'reasons', 'to', 'and', 'reading', 'and', 'are', 'in', 'of', 'scenes', 'and', 'and', 'of', 'and', 'out', 'compared', 'not', 'boss', 'yes', 'to', 'and', 'show', 'its', 'disappointed', 'fact', 'raw', 'to', 'it', 'justice', 'by', 'br', 'of', 'where', 'clear', 'fact', 'many', 'your', 'way', 'and', 'with', 'city', 'nice', 'are', 'is', 'along', 'wrong', 'not', 'as', 'it', 'way', 'she', 'but', 'this', 'anything', 'up', "haven't", 'been', 'by', 'who', 'of', 'choices', 'br', 'of', 'you', 'to', 'as', 'this', "i'd", 'it', 'and', 'who', 'of', 'shot', "you'll", 'to', 'love', 'for', 'and', 'of', 'you', 'it', 'is', 'sequels', 'of', 'little', 'quest', 'are', 'seen', 'watched', 'front', 'chemistry', 'to', 'simply', 'alive', 'of', 'chris', 'being', 'it', 'is', 'say', 'easy', 'and', 'cry', 'in', 'chemistr

In [5]:
#maximum length of a review
print('Maximum lenght of a review: {}'.format(len(max((X_train + X_test), key=len))))

Maximum lenght of a review: 2697


In [6]:
#minimum length of a review
print('Minimum lenght of a review: {}'.format(len(min((X_test + X_test), key=len))))

Minimum lenght of a review: 14


In [7]:
#to limit the max length of a review to max_words by truncating longer reviews and padding shorter reviews with a null value (0)
from keras.preprocessing import sequence
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [8]:
#importing the model and adding the layers
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
rnn_model=Sequential()
rnn_model.add(Embedding(size, embedding_size, input_length=max_words))
rnn_model.add(LSTM(100))
rnn_model.add(Dense(1, activation='sigmoid'))
print(rnn_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
#compliling the model
rnn_model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [10]:
#fitting the model
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
rnn_model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 341s 14ms/step - loss: 0.4800 - accuracy: 0.7672 - val_loss: 0.3249 - val_accuracy: 0.8750
Epoch 2/3
24936/24936 [==============================] - 339s 14ms/step - loss: 0.3064 - accuracy: 0.8751 - val_loss: 0.2908 - val_accuracy: 0.8750
Epoch 3/3
24936/24936 [==============================] - 335s 13ms/step - loss: 0.2508 - accuracy: 0.9022 - val_loss: 0.2219 - val_accuracy: 0.9219


In [11]:
#checking the accuracy
scores = rnn_model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8764399886131287
